In [72]:
import requests
from lxml import etree
import xml.etree.ElementTree as ET
import zipfile, urllib
import pandas as pd
pd.set_option('display.max_columns', None)

In [98]:
def get_data_urls(year_range=None):
    """
        Args:
            year_range - List: Parameter will take either a 2 value list or `None`. If parameter is a list, results will be restricted to years less than the first value and greater than the second.
                Example: [2003, 2010] - Files returned : 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
                Example: [2003, 2003] - Files returned : 2003
        Returns:
            all_data_links - List: List of links filtered by the requested `year_range`
    """
    url_base = 'https://exporter.nih.gov'
    url_search = url_base + '/ExPORTER_Catalog.aspx'

    url_text = requests.get(url_search).text

    parser = etree.XMLParser(recover=True)
    root = ET.fromstring(url_text, parser=parser)
    
    #### I stole this code from the internet but it will replace the prefix that gets added
    #### There's probably a better way to do this
    for elem in root.getiterator():
        if not hasattr(elem.tag, 'find'): continue
        i = elem.tag.find('}')
        if i >= 0:
            elem.tag = elem.tag[i+1:]
    ####
    data_2019 = [i.get('href') for i in root.findall('.//tr[@class="row_bg"]/td//a')]
    data_2019 = [i for i in data_2019  if ('XMLData' in i) and ('2019' in i)]
    data_2019 = [url_base + '/' + i for i in data_2019]
    ### Range + 1900 goes from 1985 to 2018    
    years = [1900 + i for i in range(85,119)]
    years = sorted(years, reverse = True)
    fy_links = [f'https://exporter.nih.gov/XMLData/final/RePORTER_PRJ_X_FY{year}.zip' for year in years]
    fy_links

    all_data_links = data_2019 + fy_links
    ### Make a copy because we modifying in place ya'll
    all_data_loop = all_data_links.copy()
    if year_range is not None:
        for link in all_data_loop:
            link_year = re.search('FY.{4}.*\.zip', link).group()
            ### Remove '.zip' then remove any '_\d' stuff that 2019 files have
            link_year = link_year.split('.')[0].split('_')[0]
            ### Remove 'FY'
            link_year = link_year[2:]
            link_year = int(link_year)
            if (link_year < year_range[0]) or (link_year > year_range[1]):
                ### modifying in place ya'll
                all_data_links.remove(link)
                
    return all_data_links

In [141]:
def extract_xml_from_zip(zip_link, out_format = 'dict'):
    """
    Function takes in a link to a zip file with an XML inside and will return a data structure containing that data.
    Data structure can be either a dictionary or a dataframe.
    
    """
    zip_file = zip_link.split('/')[-1]
    file_name = zip_file.split('.')[0]
    xml_name = file_name + '.xml'
    with zipfile.ZipFile(urllib.request.urlretrieve(zip_link, filename=None)[0], 'r') as archive:
        xml_data = archive.read(xml_name)
    rows = []
    root = ET.fromstring(xml_data)

    projects_dict = {}
    for project in root.findall('./row')[:5]:
        project_dict = {}
        for column in project.findall('./'):
            column_header = column.tag.lower()
            project_dict[column_header] = column.text
        project_dict['data_source'] = zip_link
        projects_dict[project_dict['application_id']] = project_dict
    
    if out_format.lower() == 'df':
        projects_dict = pd.DataFrame.from_dict(projects_dict, orient='index')
    return projects_dict

In [100]:
urls = get_data_urls([2003, 2003])
urls

['https://exporter.nih.gov/XMLData/final/RePORTER_PRJ_X_FY2003.zip']

In [142]:
example_df = extract_xml_from_zip(urls[0], out_format = 'df')

In [143]:
example_df.head()

,application_id,activity,administering_ic,application_type,arra_funded,budget_start,budget_end,foa_number,full_project_num,funding_ics,fy,nih_spending_cats,org_city,org_country,org_district,org_duns,org_dept,org_fips,org_state,org_zipcode,ic_name,org_name,pis,project_terms,project_title,project_start,project_end,phr,serial_number,study_section,study_section_name,support_year,suffix,subproject_id,total_cost,total_cost_sub_project,core_project_num,cfda_code,program_officer_name,ed_inst_type,award_notice_date,data_source
6557055,6557055,C06,RR,1,None,06/15/2003,06/30/2008,None,1C06RR017514-01,NCRR:2000000\,2003,None,ANN ARBOR,UNITED STATES,15,791277940,NONE,US,MI,481091274,NATIONAL CENTER FOR RESEARCH RESOURCES,UNIVERSITY OF MICHIGAN AT ANN ARBOR,None,None,RESEARCH FACILITIES CONSTRUCTION,06/15/2003,06/30/2008,None,17514,STRB,Scientific and Technical Review Board on Biome...,1,None,None,2000000,None,C06RR017514,389,"MCCULLOUGH, WILLIE",SCHOOLS OF PHARMACY,2003-06-05,https://exporter.nih.gov/XMLData/final/RePORTE...
6557924,6557924,C06,RR,1,None,09/01/2003,06/30/2008,None,1C06RR017522-01,NCRR:2000000\,2003,None,BALTIMORE,UNITED STATES,07,003255213,NONE,US,MD,212011508,NATIONAL CENTER FOR RESEARCH RESOURCES,UNIVERSITY OF MARYLAND BALTIMORE,None,None,EXTRAMURAL RESEARCH FACILITIES CONSTRUCTION,09/01/2003,06/30/2008,None,17522,STRB,Scientific and Technical Review Board on Biome...,1,None,None,2000000,None,C06RR017522,389,"MCCULLOUGH, WILLIE",SCHOOLS OF DENTISTRY/ORAL HYGN,2003-08-14,https://exporter.nih.gov/XMLData/final/RePORTE...
6712641,6712641,C06,RR,1,None,09/15/2003,12/31/2006,None,1C06RR016510-01A1,NCRR:4000000\,2003,None,SEATTLE,UNITED STATES,07,135646524,INTERNAL MEDICINE/MEDICINE,US,WA,981959472,NATIONAL CENTER FOR RESEARCH RESOURCES,UNIVERSITY OF WASHINGTON,None,None,FACILITIES CONSTRUCTION BIOCHEMISTRY RENEWAL P...,09/15/2003,12/31/2006,None,16510,STRB,Scientific and Technical Review Board on Biome...,1,A1,None,4000000,None,C06RR016510,389,"MCCULLOUGH, WILLIE",SCHOOLS OF MEDICINE,2003-08-26,https://exporter.nih.gov/XMLData/final/RePORTE...
6718184,6718184,C06,RR,1,None,09/30/2003,09/29/2006,PA-03-40,1C06RR017515-01A1,NCRR:3096536\,2003,None,SAN ANTONIO,UNITED STATES,20,None,None,US,TX,782450549,NATIONAL CENTER FOR RESEARCH RESOURCES,SOUTHWEST FOUNDATION FOR BIOMEDICAL RES,None,None,EXTRAMULAR RESEARCH FACILITIES CONSTRUCTION,09/30/2003,09/29/2006,None,17515,STRB,Scientific and Technical Review Board on Biome...,1,A1,None,3096536,None,C06RR017515,389,"MCCULLOUGH, WILLIE",None,2003-09-19,https://exporter.nih.gov/XMLData/final/RePORTE...
6720427,6720427,C06,RR,1,None,09/15/2003,09/14/2005,None,1C06RR017309-01A1,NCRR:3369600\,2003,None,LA JOLLA,UNITED STATES,53,077758407,PHARMACOLOGY,US,CA,920930934,NATIONAL CENTER FOR RESEARCH RESOURCES,UNIVERSITY OF CALIFORNIA SAN DIEGO,None,None,FACILITIES CONSTRUCTION CENTERALIZED PHARMCOGE...,09/15/2003,09/14/2005,None,17309,STRB,Scientific and Technical Review Board on Biome...,1,A1,None,3369600,None,C06RR017309,389,"MCCULLOUGH, WILLIE",SCHOOLS OF MEDICINE,2003-08-28,https://exporter.nih.gov/XMLData/final/RePORTE...


In [144]:
example_df.columns

Index(['application_id', 'activity', 'administering_ic', 'application_type',
       'arra_funded', 'budget_start', 'budget_end', 'foa_number',
       'full_project_num', 'funding_ics', 'fy', 'nih_spending_cats',
       'org_city', 'org_country', 'org_district', 'org_duns', 'org_dept',
       'org_fips', 'org_state', 'org_zipcode', 'ic_name', 'org_name', 'pis',
       'project_terms', 'project_title', 'project_start', 'project_end', 'phr',
       'serial_number', 'study_section', 'study_section_name', 'support_year',
       'suffix', 'subproject_id', 'total_cost', 'total_cost_sub_project',
       'core_project_num', 'cfda_code', 'program_officer_name', 'ed_inst_type',
       'award_notice_date', 'data_source'],
      dtype='object')